# `Предсказание победителя в онлайн-игре Dota2`

### Подход 1: градиентный бустинг "в лоб"

> 1. Считайте таблицу с признаками из файла features.csv. Удалите признаки, связанные с итогами матча.

In [1]:
import pandas as pd
features = pd.read_csv('features.csv', index_col='match_id')
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


Признаки, связанные с итогами матча: 
* duration
* radiant_win
* tower_status_radiant
* tower_status_dire
* barracks_status_radiant
* barracks_status_dire

In [2]:
# удаление признаков, связанных с итогами матча
y = features['radiant_win']
columns = ['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire']
features.drop(columns, inplace=True, axis=1)

In [3]:
 features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


> 2. Проверьте выборку на наличие пропусков Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [4]:
# количество строк в выборке
features.shape

(97230, 102)

In [5]:
# признаки с пропусками
lst = []
res = features.count()
[lst.append([column, res[column]]) for column in features if res[column] != 97230]
lst

[['first_blood_time', 77677],
 ['first_blood_team', 77677],
 ['first_blood_player1', 77677],
 ['first_blood_player2', 53243],
 ['radiant_bottle_time', 81539],
 ['radiant_courier_time', 96538],
 ['radiant_flying_courier_time', 69751],
 ['radiant_first_ward_time', 95394],
 ['dire_bottle_time', 81087],
 ['dire_courier_time', 96554],
 ['dire_flying_courier_time', 71132],
 ['dire_first_ward_time', 95404]]

* Из 102 признаков, лишь в 12 имеются пропуски, это говорит о том, что в целом пропусков в данных не много.

* Признаки события first blood ('first_blood_time', 'first_blood_team', 'first_blood_player1', 'first_blood_player2') принимают пропущенное значение, если данное событие не успело произойти за первые 5 минут. 

* Признаки 'radiant_bottle_time', 'radiant_courier_time', 'radiant_flying_courier_time', 'radiant_first_ward_time', 'dire_bottle_time', 'dire_courier_time', 'dire_flying_courier_time','dire_first_ward_time' могут принимать пропущенное значение, так как время преобретения предмета может выходить за рамки первых 5 минут.

> 3. Замените пропуски на нули с помощью функции. (На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.)

In [6]:
X = features.fillna(0)

> 4. Какой столбец содержит целевую переменную? Запишите его название.

* radiant_win

> 5. Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам, не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [7]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve, auc
import time
import datetime

In [8]:
# Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. 
X = features.fillna(100000000000)

In [9]:
kf = KFold(n_splits=5, shuffle=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 241)

In [43]:
for n in [10, 20, 30, 40]:
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators=n)
    clf.fit(X_train, y_train)
    sc = cross_val_score(clf, X_test, y_test, cv=kf, scoring='roc_auc').mean()
    print(f'Number of trees: {n} | Score: {sc}')
    print('Time elapsed:', datetime.datetime.now() - start_time)

Number of trees: 10 | Score: 0.6698684617426023
Time elapsed: 0:00:17.209291
Number of trees: 20 | Score: 0.681608519296196
Time elapsed: 0:00:34.594902
Number of trees: 30 | Score: 0.6888296142918289
Time elapsed: 0:00:51.187659
Number of trees: 40 | Score: 0.6917404038755939
Time elapsed: 0:01:07.680865


* Классификаторы настраивались достаточно долго, при этом качество повышается с ростом количества деревьев, поэтому имеет смысл использовать большее количество деревьев.

* Меры для увеличения скорости работы: 
    1. настройка параметра max_depth, чтобы деревья не были слишком глубокими
    2. обучение и кросс-валидацию не всей выборке, а некотором случайном её подмножестве

In [46]:
# max_depth
start_time = datetime.datetime.now()
clf = GradientBoostingClassifier(n_estimators=30, max_depth=2)
clf.fit(X_train, y_train)
sc = cross_val_score(clf, X_test, y_test, cv=kf, scoring='roc_auc').mean()
print(f'Number of trees: 30 | Score: {sc}')
print('Time elapsed:', datetime.datetime.now() - start_time)

Number of trees: 30 | Score: 0.6840636649680721
Time elapsed: 0:00:34.283106


In [ ]:
# как правильно подбирать max_depth?????

### Подход 2: логистическая регрессия

> 1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [89]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np

In [96]:
# нормализация признаков
X = features.fillna(0)
c_num = [0.0001, 0.001, 0.1, 1, 10, 100, 1000]

In [91]:
def log_reg(X, c_num):
    
    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    kf = KFold(n_splits=5, shuffle=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 241)
    best = 0
    res = 0
    
    for c in c_num:
        start_time = datetime.datetime.now()
        clf = LogisticRegression(penalty='l2', C = c)
        clf.fit(X_train, y_train)
        sc = cross_val_score(clf, X_test, y_test, cv=kf, scoring='roc_auc').mean()
    
        print(f'C={c} | Score: {sc}')
        print('Time elapsed:', datetime.datetime.now() - start_time)
    
    if sc > best:
        best, res = sc, c
        
    return [f'Best C is {res} | Best score is {best}']
    

In [92]:
log_reg(X, c_num)

C=0.0001 | Score: 0.7034733980538276
Time elapsed: 0:00:00.393184
C=0.001 | Score: 0.7156672287158503
Time elapsed: 0:00:00.687714
C=0.1 | Score: 0.7160277867905357
Time elapsed: 0:00:01.143360
C=1 | Score: 0.7164201853844193
Time elapsed: 0:00:01.148859
C=10 | Score: 0.7159703195722255
Time elapsed: 0:00:01.164945
C=100 | Score: 0.715098638739893
Time elapsed: 0:00:01.231070
C=1000 | Score: 0.7161957182656945
Time elapsed: 0:00:01.276883


['Best C is 1000 | Best score is 0.7161957182656945']

> 2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [97]:
categ_column = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
X.drop(categ_column, inplace=True, axis=1)

In [98]:
log_reg(X, c_num)

C=0.0001 | Score: 0.7039411921084301
Time elapsed: 0:00:00.378228
C=0.001 | Score: 0.7153458791531906
Time elapsed: 0:00:00.603383
C=0.1 | Score: 0.7163062310991751
Time elapsed: 0:00:01.131711
C=1 | Score: 0.716789705940447
Time elapsed: 0:00:01.085851
C=10 | Score: 0.7164589561935447
Time elapsed: 0:00:01.081333
C=100 | Score: 0.7161648485951988
Time elapsed: 0:00:01.273216
C=1000 | Score: 0.7156486217108707
Time elapsed: 0:00:01.285394


['Best C is 1000 | Best score is 0.7156486217108707']

* качество улучшилось, категориальные признаки вносили некоторый "шум" в данные

> 3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре.

In [16]:
len(np.unique(features[['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']] ))


108

> 4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [17]:
# мешок слов
features = features.fillna(0)
X_pick = np.zeros((features.shape[0], 112))

for i, match_id in enumerate(features.index):
    for p in range(5):
        X_pick[i, features.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [23]:
bigdata = np.hstack((X, X_pick))

In [25]:
log_reg(bigdata, c_num)

C=0.0001 | Score: 0.7275558697153052
Time elapsed: 0:00:00.569719
C=0.001 | Score: 0.7485204676172437
Time elapsed: 0:00:01.079049
C=0.1 | Score: 0.7498803910227567
Time elapsed: 0:00:02.163732
C=1 | Score: 0.7503500215617647
Time elapsed: 0:00:02.227828
C=10 | Score: 0.7502186896212324
Time elapsed: 0:00:02.412449
C=100 | Score: 0.7487946020545803
Time elapsed: 0:00:02.297250
C=1000 | Score: 0.7501452094213159
Time elapsed: 0:00:02.304192


['Best C is 1000 | Best score is 0.7501452094213159']

> 6. Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).


In [81]:
X_test = pd.read_csv('features_test.csv', index_col='match_id')
X_train = StandardScaler().fit_transform(bigdata)

In [82]:
X_test = X_test.fillna(0)
X_test.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29.0,168.0,-54.0,0.0,3,2,2,1,16.0
10,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


In [83]:
X_pick = np.zeros((X_test.shape[0], 112))

for i, match_id in enumerate(X_test.index):
    for p in range(5):
        X_pick[i, X_test.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X_test.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
    

In [84]:
X_test.drop(categ_column, inplace=True, axis=1)
X_test = np.hstack((X_test, X_pick))
X_test = StandardScaler().fit_transform(X_test)

In [86]:
clf = LogisticRegression(C = 1000)
clf.fit(X_train, y)
pred = clf.predict_proba(X_test)[:, 1]
#  выкинуть ненужные признаки

In [87]:
[max(pred), min(pred)]

[0.9964865746656767, 0.008694500151219881]